In [ ]:
import os.path

import pygetm
import pygetm.legacy

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Load North Sea domain
getm_cases = next(filter(os.path.isdir, ('/data/kb/getm-setups', '../../../getm-setups')))
domain = pygetm.legacy.domain_from_topo(os.path.join(getm_cases, 'NorthSea/Topo/NS6nm.v01.nc'), nlev=30, z0=0.001)
sim = pygetm.Simulation(domain, runtype=pygetm.BAROTROPIC_3D, fabm='../../extern/fabm/testcases/fabm-jrc-med_ergom.yaml', log_level='ERROR')

out = sim.output_manager.add_recorder(interval=10)
out.request('med_ergom_pp')
out.request('med_ergom_fl')
out.request('med_ergom_GPP')
out.request('med_ergom_OFL');

In [ ]:
sim.fabm.model.interior_dependencies['downwelling_photosynthetic_radiative_flux'].value = 5
sim.fabm.model.interior_dependencies['temperature'].value = 15
sim.fabm.model.interior_dependencies['practical_salinity'].value = 30
sim.fabm.model.horizontal_dependencies['surface_downwelling_photosynthetic_radiative_flux'].value = 50
sim.fabm.model.horizontal_dependencies['wind_speed'].value = 50
sim.fabm.model.horizontal_dependencies['bottom_stress'].value = 0

sim.fabm.start()

In [ ]:
# Simulate for 1 day
dt = 1800
sim.output_manager.start()
for itime in range(int(86400 / dt)):
    sim.fabm.update_sources()
    sim.fabm.advance(dt)
    sim.output_manager.save(itime * dt, itime)
sim.output_manager.close(itime * dt)

In [ ]:
# Plot surface field and y-z slice 
fig, (ax1, ax2) = matplotlib.pyplot.subplots(ncols=2, figsize=(12, 4))
sim['med_ergom_pp'].isel(z=-1).plot(ax=ax1)
sim['med_ergom_OFL'].plot(ax=ax2)

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
out.x.med_ergom_pp.mean(axis=(1, 2, 3)).plot()